# Gallery Example: Er/M/1 Queue (Erlang Arrivals)

This example demonstrates an Er/M/1 queueing system:
- **Arrivals**: Erlang distribution (less variable than Poisson)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Erlang distribution (with shape parameter k=5) has lower variance than exponential, representing more regular arrivals than a Poisson process.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_erlm1(): Network {    """Create Er/M/1 queueing model"""    model = Network("Er/M/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Erlang arrival with mean=1 and order=5 (less variable than exponential)    source.setArrival(oclass, Erlang.fit_mean_and_order(1, 5))    # Exponential service with rate=2 (mean service time = 0.5)    queue.setService(oclass, Exp(2))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_erlm1()

## Theoretical Analysis for Er/M/1

For Er/M/1 with:
- **Arrival rate**: λ = 1 (Erlang with mean=1, shape=5)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

The Erlang distribution characteristics:
- **Mean**: 1.0
- **Variance**: 1/5 = 0.2 (lower than exponential variance = 1)
- **Coefficient of Variation**: C²ₐ = 1/5 = 0.2

This lower arrival variability typically results in better performance than M/M/1.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare Er/M/1 with M/M/1
println("\n=== Comparison with M/M/1 ===")
// Create equivalent M/M/1 model
fun create_mm1_equivalent(): Network {
    val model_mm1 = Network("M/M/1-Equivalent")
    val source = Source(model_mm1, "Source")
    val queue = Queue(model_mm1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_mm1, "Sink")
    
    val oclass = OpenClass(model_mm1, "Class")
    source.setArrival(oclass, Exp(1))  # Same mean arrival rate
    queue.setService(oclass, Exp(2))   # Same service rate
    
    val P = model_mm1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

val model_mm1 = create_mm1_equivalent()
val solver_erlm1 = MVA(model)
val solver_mm1 = MVA(model_mm1)

val avg_table_erlm1 = solver_erlm1.avgTable
val avg_table_mm1 = solver_mm1.avgTable

println("Er/M/1 Results:")
println(avg_table_erlm1)

println("\nM/M/1 Results:")
println(avg_table_mm1)
// Extract key metrics for comparison
val erlm1_resp = float(avg_table_erlm1.iloc[1, 2])  # Queue response time
val erlm1_length = float(avg_table_erlm1.iloc[1, 3])  # Queue length

val mm1_resp = float(avg_table_mm1.iloc[1, 2])  # Queue response time
val mm1_length = float(avg_table_mm1.iloc[1, 3])  # Queue length

println(f"\nPerformance Comparison:")
println(f"Response Time: Er/M/1={erlm1_resp:.4f}, M/M/1={mm1_resp:.4f}")
println(f"Queue Length: Er/M/1={erlm1_length:.4f}, M/M/1={mm1_length:.4f}")
println(f"Improvement: {((mm1_resp - erlm1_resp) / mm1_resp * 100):.1f}% better response time with Erlang arrivals")

In [ ]:
// Analyze impact of Erlang order parameter
println("\n=== Impact of Erlang Order Parameter ===")

fun create_erlang_model(order): Network {
    """Create Er/M/1 model with specified Erlang order"""
    val model_erl = Network(f"Er{order}/M/1")
    val source = Source(model_erl, "Source")
    val queue = Queue(model_erl, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_erl, "Sink")
    
    val oclass = OpenClass(model_erl, "Class")
    source.setArrival(oclass, Erlang.fit_mean_and_order(1, order))
    queue.setService(oclass, Exp(2))
    
    val P = model_erl.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl
// Test different Erlang orders
val orders = [1, 2, 3, 5, 10]  # Order 1 = Exponential

println("Order | CV² | Response Time | Queue Length")
println("-" * 45)

for order in orders:
    val model_order = create_erlang_model(order)
    val solver = MVA(model_order)
    val avg_table = solver.avgTable
    
    val resp_time = float(avg_table.iloc[1, 2])
    val queue_length = float(avg_table.iloc[1, 3])
    val cv_squared = 1.0 / order  # Coefficient of variation squared for Erlang
    
    println(f"  {order:2d}  | {cv_squared:.3f} |     {resp_time:.4f}    |    {queue_length:.4f}")

println("\nNote: Higher order (lower CV²) leads to better performance due to reduced arrival variability.")